In [1]:
import tensorflow as tf 
from tensorflow import keras
import tensorflow_datasets as tfds

from matplotlib import pyplot as plt
from matplotlib import ticker
from pathlib import Path

from datetime import datetime

import sys
sys.path.insert(0, "..")

print(tf.__version__)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
    # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
        print(e)

2.2.0
2 Physical GPUs, 2 Logical GPUs


## Load Data

In [2]:
## Download file
data_path = Path("..") / "datasets" / "data"
if not data_path.is_dir():
    data_path.mkdir(parents=True)

In [3]:
datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True, data_dir=data_path)
mnist_train, mnist_test = datasets['train'], datasets['test']

In [5]:
# define model
num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples
BUFFER_SIZE = 60000
BATCH_SIZE = 32
print(f"Training data samples: {num_train_examples}, Testing data samples: {num_test_examples}")

Training data samples: 60000, Testing data samples: 10000


In [6]:
# Network Parameters
num_input = 1 # MNIST data input (img shape: 28*28)
timesteps = 28 * 28 # timesteps
num_hidden = 128 # hidden layer num of features
num_classes = 10 # MNIST total classes (0-9 digits)

In [7]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    image = tf.reshape(image, (-1, 1))
    return image, label

In [8]:
train_dataset = mnist_train.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

In [9]:
logdir = "../logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

## Create Model Using Neuromodulated Bistable RNNs

In [4]:
from bistablernn import NBR

In [10]:
model = tf.keras.Sequential([
  NBR(units=num_hidden, input_shape=(28*28, num_input), use_bias=True, 
                   recurrent_dropout=0, unroll=False, activation = "tanh", 
                   recurrent_activation = "sigmoid"),
  tf.keras.layers.Dense(num_classes)
])

model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.002, beta_1=0.1),
            metrics=['accuracy'])

In [ ]:
model.fit(train_dataset, epochs=100, validation_data=eval_dataset, callbacks=[tensorboard_callback])

Epoch 1/100
1875/1875 [==============================] - 86s 46ms/step - loss: 2.1445 - accuracy: 0.1874 - val_loss: 2.3245 - val_accuracy: 0.1245
Epoch 2/100
1875/1875 [==============================] - 85s 45ms/step - loss: 2.3040 - accuracy: 0.1217 - val_loss: 2.3256 - val_accuracy: 0.0892
Epoch 3/100
1875/1875 [==============================] - 85s 45ms/step - loss: 2.3209 - accuracy: 0.1060 - val_loss: 2.3528 - val_accuracy: 0.0421
Epoch 4/100
1875/1875 [==============================] - 84s 45ms/step - loss: 2.3108 - accuracy: 0.1126 - val_loss: 2.3159 - val_accuracy: 0.0929
Epoch 5/100
1875/1875 [==============================] - 85s 45ms/step - loss: 2.3020 - accuracy: 0.1129 - val_loss: 2.3070 - val_accuracy: 0.1021
Epoch 6/100
1875/1875 [==============================] - 85s 45ms/step - loss: 2.3111 - accuracy: 0.1128 - val_loss: 2.2876 - val_accuracy: 0.1061
Epoch 7/100
1875/1875 [==============================] - 85s 45ms/step - loss: 1.9906 - accuracy: 0.2325 - val_loss: 1

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, 128)               50304     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 51,594
Trainable params: 51,594
Non-trainable params: 0
_________________________________________________________________
